# Day 16: The Floor Will Be Lava
With the beam of light completely focused somewhere, the reindeer leads you deeper still into the Lava Production Facility. At some point, you realize that the steel facility walls have been replaced with cave, and the doorways are just cave, and the floor is cave, and you're pretty sure this is actually just a giant cave.

Finally, as you approach what must be the heart of the mountain, you see a bright light in a cavern up ahead. There, you discover that the beam of light you so carefully focused is emerging from the cavern wall closest to the facility and pouring all of its energy into a contraption on the opposite side.

Upon closer inspection, the contraption appears to be a flat, two-dimensional square grid containing empty space (.), mirrors (/ and `\`), and splitters (| and -).

The contraption is aligned so that most of the beam bounces around the grid, but each tile on the grid converts some of the beam's light into heat to melt the rock in the cavern.

You note the layout of the contraption (your puzzle input). For example:
```
.|...\....
|.-.\.....
.....|-...
........|.
..........
.........\
..../.\\..
.-.-/..|..
.|....-|.\
..//.|....
```
The beam enters in the top-left corner from the left and heading to the right. Then, its behavior depends on what it encounters as it moves:

If the beam encounters empty space (.), it continues in the same direction.
If the beam encounters a mirror (/ or `\`), the beam is reflected 90 degrees depending on the angle of the mirror. For instance, a rightward-moving beam that encounters a / mirror would continue upward in the mirror's column, while a rightward-moving beam that encounters a \ mirror would continue downward from the mirror's column.
If the beam encounters the pointy end of a splitter (| or -), the beam passes through the splitter as if the splitter were empty space. For instance, a rightward-moving beam that encounters a - splitter would continue in the same direction.
If the beam encounters the flat side of a splitter (| or -), the beam is split into two beams going in each of the two directions the splitter's pointy ends are pointing. For instance, a rightward-moving beam that encounters a | splitter would split into two beams: one that continues upward from the splitter's column and one that continues downward from the splitter's column.
Beams do not interact with other beams; a tile can have many beams passing through it at the same time. A tile is energized if that tile has at least one beam pass through it, reflect in it, or split in it.

In the above example, here is how the beam of light bounces around the contraption:
```
>|<<<\....
|v-.\^....
.v...|->>>
.v...v^.|.
.v...v^...
.v...v^..\
.v../2\\..
<->-/vv|..
.|<<<2-|.\
.v//.|.v..
```
Beams are only shown on empty tiles; arrows indicate the direction of the beams. If a tile contains beams moving in multiple directions, the number of distinct directions is shown instead. Here is the same diagram but instead only showing whether a tile is energized (#) or not (.):
```
######....
.#...#....
.#...#####
.#...##...
.#...##...
.#...##...
.#..####..
########..
.#######..
.#...#.#..
```
Ultimately, in this example, 46 tiles become energized.

The light isn't energizing enough tiles to produce lava; to debug the contraption, you need to start by analyzing the current situation. With the beam starting in the top-left heading right, how many tiles end up being energized?

In [16]:
from collections import defaultdict

revertDirection = {
        'L': 'R',
        'R': 'L',
        'U': 'D',
        'D': 'U'
    }
indexShift = {
        'L': [0,-1],
        'R': [0,1],
        'U': [-1,0],
        'D': [1,0],
    }
nextDirection = {
    '.': {
        'L': ['R'],
        'R': ['L'],
        'U': ['D'],
        'D': ['U']
    },
    '|': {
        'L': ['U','D'],
        'R': ['U','D'],
        'U': ['D'],
        'D': ['U']
    },
    '-': {
        'L': ['R'],
        'R': ['L'],
        'U': ['R','L'],
        'D': ['R','L'],
    },
    '/': {
        'L': ['U'],
        'R': ['D'],
        'U': ['L'],
        'D': ['R']
    },
    '\\': {
        'L': ['D'],
        'R': ['U'],
        'U': ['R'],
        'D': ['L']
    },
}

def result(inputfile):

    input = []
    foundDirs = defaultdict(lambda: defaultdict(lambda: []))
    enegized = defaultdict(lambda: defaultdict(lambda: 0))

    with open(inputfile) as f:
        for line in f:
            input.append([*line.strip()])
    def travelThroughPoint(x,y,comingFrom):
        if 0<= x < len(input) and 0<= y < len(input[0]):
            enegized[x][y] = 1
            nextDirs = nextDirection[input[x][y]][comingFrom]
            foundDirs[x][y].append(comingFrom)
            for dir in nextDirs:
                if dir not in foundDirs[x][y]:
                    foundDirs[x][y].append(dir)
                    nextComingFrom = revertDirection[dir]
                    shift = indexShift[dir]
                    nextX = x + shift[0]
                    nextY = y + shift[1]
                    travelThroughPoint(nextX, nextY, nextComingFrom)
    travelThroughPoint(0,0,'L')
    return sum([sum(line.values()) for line in enegized.values()])


In [17]:
result("example-input.txt")

46

In [18]:
result("input.txt")

7434

### Part Two
As you try to work out what might be wrong, the reindeer tugs on your shirt and leads you to a nearby control panel. There, a collection of buttons lets you align the contraption so that the beam enters from any edge tile and heading away from that edge. (You can choose either of two directions for the beam if it starts on a corner; for instance, if the beam starts in the bottom-right corner, it can start heading either left or upward.)

So, the beam could start on any tile in the top row (heading downward), any tile in the bottom row (heading upward), any tile in the leftmost column (heading right), or any tile in the rightmost column (heading left). To produce lava, you need to find the configuration that energizes as many tiles as possible.

In the above example, this can be achieved by starting the beam in the fourth tile from the left in the top row:
```
.|<2<\....
|v-v\^....
.v.v.|->>>
.v.v.v^.|.
.v.v.v^...
.v.v.v^..\
.v.v/2\\..
<-2-/vv|..
.|<<<2-|.\
.v//.|.v..
```
Using this configuration, 51 tiles are energized:
```
.#####....
.#.#.#....
.#.#.#####
.#.#.##...
.#.#.##...
.#.#.##...
.#.#####..
########..
.#######..
.#...#.#..
```
Find the initial beam configuration that energizes the largest number of tiles; how many tiles are energized in that configuration?

In [19]:
def scoreOfEntry(input,startLine,startColumn,startDirection):
    foundDirs = defaultdict(lambda: defaultdict(lambda: []))
    enegized = defaultdict(lambda: defaultdict(lambda: 0))

    def travelThroughPoint(x,y,comingFrom):
        if 0<= x < len(input) and 0<= y < len(input[0]):
            enegized[x][y] = 1
            nextDirs = nextDirection[input[x][y]][comingFrom]
            foundDirs[x][y].append(comingFrom)
            for dir in nextDirs:
                if dir not in foundDirs[x][y]:
                    foundDirs[x][y].append(dir)
                    nextComingFrom = revertDirection[dir]
                    shift = indexShift[dir]
                    nextX = x + shift[0]
                    nextY = y + shift[1]
                    travelThroughPoint(nextX, nextY, nextComingFrom)
    travelThroughPoint(startLine,startColumn,startDirection)
    return sum([sum(line.values()) for line in enegized.values()])

def result2(inputfile):

    input = []
    scores = []

    with open(inputfile) as f:
        for line in f:
            input.append([*line.strip()])
    length = len(input)
    width = len(input[0])
    for i in range(length):
        scores.append(scoreOfEntry(input,i,0,'L'))
        scores.append(scoreOfEntry(input,i,width-1,'R'))
    for i in range(width):
        scores.append(scoreOfEntry(input,0,i,'U'))
        scores.append(scoreOfEntry(input,length-1,i,'D'))
    return max(scores)

In [21]:
result2("example-input.txt")

51

In [22]:
result2("input.txt")

8183